<a href="https://colab.research.google.com/github/Ziaad03/ngram-language-model-sentiment-classification/blob/main/NaiveBayesClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

from datasets import load_dataset
import numpy as np
from collections import Counter

# This downloads the dataset and loads its splits (e.g., train, validation, test)
dataset = load_dataset("stanfordnlp/sst")

# Print details about the dataset splits
print(dataset)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.68k [00:00<?, ?B/s]

sst.py:   0%|          | 0.00/9.13k [00:00<?, ?B/s]

The repository for stanfordnlp/sst contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/stanfordnlp/sst.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})


In [3]:
# Print train
dataset['train']

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree'],
    num_rows: 8544
})

In [4]:
print("Columns:", dataset["train"].column_names)
print("Features:", dataset["train"].features)


Columns: ['sentence', 'label', 'tokens', 'tree']
Features: {'sentence': Value(dtype='string', id=None), 'label': Value(dtype='float32', id=None), 'tokens': Value(dtype='string', id=None), 'tree': Value(dtype='string', id=None)}


In [ ]:
print(dataset["train"][0])


{'sentence': "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .", 'label': 0.6944400072097778, 'tokens': "The|Rock|is|destined|to|be|the|21st|Century|'s|new|``|Conan|''|and|that|he|'s|going|to|make|a|splash|even|greater|than|Arnold|Schwarzenegger|,|Jean-Claud|Van|Damme|or|Steven|Segal|.", 'tree': '70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|55|54|53|52|51|49|47|47|46|46|45|40|40|41|39|38|38|43|37|37|69|44|39|42|41|42|43|44|45|50|48|48|49|50|51|52|53|54|55|66|57|59|59|60|61|62|63|64|65|66|67|68|69|71|71|0'}


In [ ]:
df = dataset["train"].to_pandas()
print(df.head())


                                            sentence    label  \
0  The Rock is destined to be the 21st Century 's...  0.69444   
1  The gorgeously elaborate continuation of `` Th...  0.83333   
2  Singer\/composer Bryan Adams contributes a sle...  0.62500   
3  You 'd think by now America would have had eno...  0.50000   
4               Yet the act is still charming here .  0.72222   

                                              tokens  \
0  The|Rock|is|destined|to|be|the|21st|Century|'s...   
1  The|gorgeously|elaborate|continuation|of|``|Th...   
2  Singer\/composer|Bryan|Adams|contributes|a|sle...   
3  You|'d|think|by|now|America|would|have|had|eno...   
4               Yet|the|act|is|still|charming|here|.   

                                                tree  
0  70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|5...  
1  71|70|69|69|67|67|66|64|63|62|62|61|61|58|57|5...  
2  72|71|71|70|68|68|67|67|66|63|62|62|60|60|58|5...  
3  36|35|34|33|33|32|30|29|27|26|25|24|23|23|22|2...

In [ ]:
print("Number of training samples:", len(dataset["train"]))
print("Number of validation samples:", len(dataset["validation"]))
print("Number of test samples:", len(dataset["test"]))

Number of training samples: 8544
Number of validation samples: 1101
Number of test samples: 2210


In [5]:
#extract the data
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

#print
print("Train data:", train_data[0])
print("Validation data:", validation_data[0])
print("Test data:", test_data[0])

Train data: {'sentence': "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .", 'label': 0.6944400072097778, 'tokens': "The|Rock|is|destined|to|be|the|21st|Century|'s|new|``|Conan|''|and|that|he|'s|going|to|make|a|splash|even|greater|than|Arnold|Schwarzenegger|,|Jean-Claud|Van|Damme|or|Steven|Segal|.", 'tree': '70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|55|54|53|52|51|49|47|47|46|46|45|40|40|41|39|38|38|43|37|37|69|44|39|42|41|42|43|44|45|50|48|48|49|50|51|52|53|54|55|66|57|59|59|60|61|62|63|64|65|66|67|68|69|71|71|0'}
Validation data: {'sentence': "It 's a lovely film with lovely performances by Buy and Accorsi .", 'label': 0.791670024394989, 'tokens': "It|'s|a|lovely|film|with|lovely|performances|by|Buy|and|Accorsi|.", 'tree': '25|23|21|20|20|19|17|17|16|14|14|15|24|15|16|18|18|19|22|21|22|23|24|25|0'}
Test data: {'sentence': 'Effective but too-tepid bio

# **Preprocessing for Classification**

In [ ]:

print(train_data)

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree'],
    num_rows: 8544
})


In [6]:
def map_labels(data):
    label = data["label"]
    if 0 <= label <= 0.2:
        data["label"] = 0
    elif 0.2 < label <= 0.4:
        data["label"] = 1
    elif 0.4 < label <= 0.6:
        data["label"] = 2
    elif 0.6 < label <= 0.8:
        data["label"] = 3
    elif 0.8 < label <= 1:
        data["label"] = 4
    return data

train_data = train_data.map(map_labels)
validation_data =validation_data.map(map_labels)
test_data = test_data.map(map_labels)

# Check a few records
df = train_data.to_pandas()
print(df.head())

df = validation_data.to_pandas()
print(df.head())

df = test_data.to_pandas()
print(df.head())


Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

                                            sentence  label  \
0  The Rock is destined to be the 21st Century 's...    3.0   
1  The gorgeously elaborate continuation of `` Th...    4.0   
2  Singer\/composer Bryan Adams contributes a sle...    3.0   
3  You 'd think by now America would have had eno...    2.0   
4               Yet the act is still charming here .    3.0   

                                              tokens  \
0  The|Rock|is|destined|to|be|the|21st|Century|'s...   
1  The|gorgeously|elaborate|continuation|of|``|Th...   
2  Singer\/composer|Bryan|Adams|contributes|a|sle...   
3  You|'d|think|by|now|America|would|have|had|eno...   
4               Yet|the|act|is|still|charming|here|.   

                                                tree  
0  70|70|68|67|63|62|61|60|58|58|57|56|56|64|65|5...  
1  71|70|69|69|67|67|66|64|63|62|62|61|61|58|57|5...  
2  72|71|71|70|68|68|67|67|66|63|62|62|60|60|58|5...  
3  36|35|34|33|33|32|30|29|27|26|25|24|23|23|22|2...  
4        

In [7]:
filtered_dataset = train_data.filter(lambda example: example["label"] == 2)
docsClassNumber = filtered_dataset
print(docsClassNumber)


Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree'],
    num_rows: 1624
})


In [8]:
#get the labels
train_labels = train_data["label"]
validation_labels = validation_data["label"]
test_labels = test_data["label"]


print("Train labels:", train_labels)
print("Validation labels:", validation_labels)
print("Test labels:", test_labels)

Train labels: [3.0, 4.0, 3.0, 2.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 2.0, 4.0, 2.0, 4.0, 3.0, 3.0, 2.0, 4.0, 3.0, 4.0, 3.0, 2.0, 4.0, 2.0, 2.0, 4.0, 3.0, 4.0, 1.0, 4.0, 3.0, 2.0, 3.0, 4.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 4.0, 2.0, 3.0, 2.0, 0.0, 3.0, 3.0, 1.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0, 1.0, 1.0, 4.0, 3.0, 4.0, 4.0, 3.0, 2.0, 1.0, 3.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 2.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 1.0, 4.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, 3.0, 3.0, 3.0, 4.0, 1.0, 2.0, 2.0, 2.0, 4.0, 3.0, 3.0, 3.0, 3.0, 2.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 2.0, 4.0, 3.0, 2.0, 1.0, 3.0, 2.0, 3.0, 3.0, 1.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 2.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0, 2.0, 4.0, 3.0, 1.0, 

# **Naive Bayes Classification**

In [9]:
def get_vocabulary(data):

    vocabs = set()

    for token_sequence in data['tokens']:
        word_chars = []
        for token in token_sequence:
            if token == "|":
                # Join the characters into a full word.
                if word_chars:
                    word_str = ''.join(word_chars)
                    # Append if the word is longer than one character.
                    if len(word_str) > 1:
                        vocabs.add(word_str)
                    word_chars = []
            else:
                # Only append if the token is alphabetic or numeric.
                if token.isalpha() or token.isdigit():
                    word_chars.append(token)
        # In case the sequence doesn't end with a delimiter.
        if word_chars:
            word_str = ''.join(word_chars)
            if len(word_str) > 1:
                vocabs.add(word_str)

    return vocabs





In [10]:
print(get_vocabulary(train_data))
print(len(get_vocabulary(train_data)))

{'raises', 'Memories', 'zest', 'banality', 'Frankie', 'Oprah', 'new', 'blush', 'hoods', 'richness', 'Crystal', 'Com', 'beseechingly', 'instincts', 'producers', 'subjected', 'affords', 'hankypanky', 'Said', 'purer', 'like', 'possible', 'pathetic', 'mired', 'illtimed', 'scathing', 'reopens', 'angstridden', 'worn', 'taps', 'accidental', 'lasting', 'imagination', 'doctorate', 'bookontape', 'verbal', 'Confuses', 'timelessness', 'image', 'Film', 'naturalseeming', 'Bros', 'flashbulbs', 'Nation', 'MGM', 'intractable', 'artist', 'cheapshot', 'peevish', 'punny', 'gobbler', 'thrillers', 'Becker', 'compromise', 'downanddirty', 'sustenance', 'irrigates', 'rank', 'telephone', 'Mild', 'Pollak', 'English', 'jerking', 'preWWII', 'RINGING', 'Swimfan', 'buttons', 'seductively', 'compliment', 'strangest', 'workers', 'asylum', 'street', 'hustlers', 'screwy', 'ultimately', 'Luther', 'leavened', 'iteration', 'Superb', 'teengang', 'Proves', 'Bearable', 'Americans', 'supplies', 'collaboration', 'blubber', 'Ein

In [ ]:

# def get_vocabulary(train_data):
#     """
#     Extract unique words from the 'tokens' field of the dataset.

#     Args:
#         train_data (Dataset): The 'train' split.

#     Returns:
#         set: Unique words (lowercased) from all documents.
#     """
#     vocab = set()
#     for doc in train_data:
#         # Split 'tokens' into words and lowercase them
#         tokens = doc['tokens'].split('|')
#         for token in tokens:
#             if token:  # Skip empty tokens
#                 vocab.add(token.lower())
#     return sorted(vocab)

# print(get_vocabulary(train_data))
# print(len(get_vocabulary(train_data)))

['!', '!?', '#', '$', '&', "'", "''", "'30s", "'40s", "'50s", "'53", "'60s", "'70s", "'80s", "'90s", "'d", "'em", "'ll", "'m", "'n", "'n'", "'re", "'s", "'til", "'ve", '(', ')', '+', ',', '-', '--', '.', '...', '1', '1.2', '1.8', '10', '10,000', '10-course', '10-year', '10-year-old', '100', '100-minute', '100-year', '101', '102-minute', '103-minute', '104', '105', '10th', '10th-grade', '11', '110', '112-minute', '12', '12-year-old', '120', '127', '129-minute', '12th', '13', '13th', '14-year-old', '140', '146', '15', '15-year', '15th', '163', '168-minute', '170', '1790', '18', '18-year-old', '1899', '19', '1915', '1920', '1930s', '1933', '1937', '1938', '1940s', '1950', '1950s', '1952', '1953', '1957', '1958', '1959', '1960', '1960s', '1962', '1970', '1970s', '1971', '1972', '1973', '1975', '1979', '1980', '1980s', '1984', '1986', '1987', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1997', '1998', '1999', '19th', '19th-century', '1\\/2', '2', '2,500', '2-day', '20', '20,000'

In [11]:
def getWordOccurenceInClassDocs(word, classDocs):

    total_count = 0
    word = word.lower()  # Convert word to lowercase for case-insensitive matching

    for doc in classDocs:
        tokens_str = doc['tokens']  # Get the 'tokens' string (e.g., "The|Rock|is|destined")
        words = tokens_str.lower().split('|')  # Split into list of words, lowercase
        total_count += words.count(word)  # Add count of word in this document

    return total_count

In [12]:
def naive_bayes_classifier(train_data, classes):

    # Total number of documents
    docs_number = len(train_data)

    # Get the vocabulary (unique words) from the 'tokens' field
    vocab = get_vocabulary(train_data)

    # Initialize dictionaries for probabilities
    log_prior = {}
    loglikelihood = {}

    # Process each class
    for c in classes:
        # Filter documents belonging to class c
        class_docs = train_data.filter(lambda example: example['label'] == c)
        docs_class_number = len(class_docs)

        # Compute log prior: log(P(c)) = log(#docs in c / total #docs)
        log_prior[c] = np.log(docs_class_number / docs_number) if docs_class_number > 0 else -np.inf

        # Count word frequencies in this class
        word_freq = Counter()
        for doc in class_docs:
            # Split 'tokens' into words and convert to lowercase for consistency
            words = doc['tokens'].lower().split('|')
            word_freq.update(words)

        # Total number of word occurrences in this class
        total_word_occurrences = sum(word_freq.values())

        # Compute log likelihood for each word in the vocabulary
        loglikelihood[c] = {}
        for word in vocab:
            # Get word count in this class (case-insensitive)
            count = word_freq.get(word.lower(), 0)
            # Compute P(word|c) with add-one (Laplace) smoothing and take log
            loglikelihood[c][word] = np.log((count + 1) / (total_word_occurrences + len(vocab)))

    return log_prior, loglikelihood, vocab

# **Train the classfier**

In [13]:
# Train
log_prior, loglikelihood, vocab = naive_bayes_classifier(train_data, [0, 1, 2, 3, 4])
print(log_prior)
print(loglikelihood)
print(vocab)

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8544 [00:00<?, ? examples/s]

{0: -2.0572184048951256, 1: -1.3486233932896636, 2: -1.6603370404783528, 3: -1.3028003989421393, 4: -1.892138654535677}
{0: {'raises': -10.572572546745315, 'Memories': -9.87942536618537, 'zest': -10.572572546745315, 'banality': -9.87942536618537, 'Frankie': -10.572572546745315, 'Oprah': -9.87942536618537, 'new': -7.6822007888491495, 'blush': -10.572572546745315, 'hoods': -10.572572546745315, 'richness': -10.572572546745315, 'Crystal': -10.572572546745315, 'Com': -9.87942536618537, 'beseechingly': -10.572572546745315, 'instincts': -10.572572546745315, 'producers': -9.473960258077204, 'subjected': -10.572572546745315, 'affords': -10.572572546745315, 'hankypanky': -10.572572546745315, 'Said': -9.473960258077204, 'purer': -10.572572546745315, 'like': -6.268507453541145, 'possible': -8.493131005065479, 'pathetic': -8.493131005065479, 'mired': -9.87942536618537, 'illtimed': -10.572572546745315, 'scathing': -10.572572546745315, 'reopens': -10.572572546745315, 'angstridden': -10.57257254674531

# **Test the classfier**

In [14]:
def test_naive_bayes(testdoc, logprior, loglikelihood, C, V):

    sum_scores = {}

    #  iterate over each class
    for c in C:
        sum_scores[c] = logprior[c]  # Start with the log prior for this class
        #  process each word in the document for this class
        for word in testdoc:
            if word in V:
                sum_scores[c] += loglikelihood[c][word]  # Add log likelihood

    # Predict the class with the highest score
    predicted_class = max(sum_scores, key=sum_scores.get)
    return predicted_class

In [15]:
print(test_data[0]['tokens'].split('|'))
predicted_class = test_naive_bayes(test_data[0]['tokens'].split('|'), log_prior, loglikelihood, [0, 1, 2, 3, 4], vocab)
print(predicted_class)

['Effective', 'but', 'too-tepid', 'biopic']
3


# **Comparison with scikit-learn**

In [16]:
test_tokens = [doc["tokens"] for doc in test_data]
train_tokens = [doc["tokens"] for doc in train_data]
# Convert tokens to space-separated strings
test_texts = [" ".join(tokens.split("|")) for tokens in test_tokens]
train_texts = [" ".join(tokens.split("|")) for tokens in train_tokens]

print(test_texts)

['Effective but too-tepid biopic', 'If you sometimes like to go to the movies to have fun , Wasabi is a good place to start .', "Emerges as something rare , an issue movie that 's so honest and keenly observed that it does n't feel like one .", 'The film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .', 'Offers that rare combination of entertainment and education .', 'Perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .', "Steers turns in a snappy screenplay that curls at the edges ; it 's so clever you want to hate it .", 'But he somehow pulls it off .', 'Take Care of My Cat offers a refreshingly different slice of Asian cinema .', 'This is a film well worth seeing , talking and singing heads and all .', 'What really surprises about Wisegirls is its low-key quality and genuine tenderness .', '( Wendigo is ) why we go to the cinema : to be fed throug

In [22]:
FinalPrediction = []
for text in test_texts:

    predicted_class = test_naive_bayes(text.split(' '), log_prior, loglikelihood, [0, 1, 2, 3, 4], vocab)
    FinalPrediction.append(predicted_class)

print(FinalPrediction)


[3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 4, 1, 1, 1, 4, 4, 3, 4, 3, 4, 3, 1, 3, 3, 2, 3, 1, 2, 4, 3, 1, 3, 3, 3, 3, 3, 3, 1, 2, 1, 3, 3, 2, 1, 3, 3, 3, 3, 4, 3, 1, 3, 3, 3, 3, 3, 3, 1, 2, 4, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 3, 1, 3, 3, 3, 4, 3, 1, 3, 3, 3, 2, 3, 1, 1, 4, 1, 3, 3, 3, 0, 3, 3, 3, 2, 1, 3, 3, 3, 3, 3, 3, 1, 3, 4, 3, 1, 3, 4, 1, 3, 3, 3, 3, 3, 1, 1, 3, 1, 3, 4, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 3, 1, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 1, 3, 4, 3, 0, 3, 4, 1, 3, 2, 1, 3, 3, 3, 3, 3, 1, 3, 2, 3, 4, 3, 4, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 4, 3, 1, 3, 3, 3, 3, 3, 4, 3, 2, 1, 1, 1, 3, 1, 3, 3, 1, 2, 3, 1, 3, 3, 3, 1, 1, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 1, 3, 3, 4, 3, 2, 3, 1, 1, 3, 2, 1, 3, 3, 4, 3, 1, 3, 3, 4, 3, 0, 3, 1, 3, 2, 3, 3, 3, 3, 1, 1, 4, 3, 1, 1, 3, 3, 1, 4, 3, 3, 1, 

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

vectorizer = CountVectorizer(lowercase=True, token_pattern=r"\b\w+\b")

X_train = vectorizer.fit_transform(train_texts)
x_test = vectorizer.transform(test_texts)
# X_test is now a sparse matrix of word counts

from sklearn.naive_bayes import MultinomialNB

# Train
clf = MultinomialNB(alpha=1.0)  # Laplace smoothing
clf.fit(X_train, train_labels)

# Predict on the same data
predictions = clf.predict(x_test)
print(predictions)
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions, average='weighted')
recall = recall_score(test_labels, predictions, average='weighted')
f1 = f1_score(test_labels, predictions, average='weighted')

implementedAccuracy = accuracy_score(test_labels, FinalPrediction)
implementedPrecision = precision_score(test_labels, FinalPrediction, average='weighted')
implementedRecall = recall_score(test_labels, FinalPrediction, average='weighted')
implementedF1 = f1_score(test_labels, FinalPrediction, average='weighted')




# Print results for comparison
print(f"Scikit-learn Accuracy: {accuracy}")
print(f"Scikit-learn Precision: {precision}")
print(f"Scikit-learn Recall: {recall}")
print(f"Scikit-learn F1-Score: {f1}")

print("----------------------------------------------")
#Print implemented results
print(f"Implemented Accuracy: {implementedAccuracy}")
print(f"Implemented Precision: {implementedPrecision}")
print(f"Implemented Recall: {implementedRecall}")
print(f"Implemented F1-Score: {implementedF1}")


[2. 3. 3. ... 4. 1. 1.]
Scikit-learn Accuracy: 0.40452488687782806
Scikit-learn Precision: 0.40972715492701367
Scikit-learn Recall: 0.40452488687782806
Scikit-learn F1-Score: 0.3565956313574668
Implemented Accuracy: 0.4081447963800905
Implemented Precision: 0.4167727476110429
Implemented Recall: 0.4081447963800905
Implemented F1-Score: 0.35919337951368563


# **Confusion Matrix implementation**

In [35]:
def im_confusion_matrix(true,pred):
  classes = set(true + pred)
  print(classes)
  num_classes = 5
  shape = (num_classes, num_classes)
  mat = np.zeros(shape)
  n = max(len(true),len(pred))
  for i in range(num_classes):
    for j in range(num_classes):
      for k in range(n):
        if true[k]-1 == i:
          if pred[k]-1 == j:
            mat[i][j] = mat[i][j] + 1
  return mat

In [25]:


def getPrecisionPerClass(matrix, label):
    index = label - 1  # Map label (e.g., 1) to index (e.g., 0)
    if index < 0 or index >= matrix.shape[0]:
        raise ValueError(f"Label {label} is out of range for matrix with {matrix.shape[0]} classes")
    TP = matrix[index, index]
    FP = np.sum(matrix[:, index]) - TP
    denom = TP + FP
    if denom == 0:
        return 0.0
    return TP / denom


In [26]:
def getRecallPerClass(matrix, label):
    index = label - 1
    if index < 0 or index >= matrix.shape[0]:
        raise ValueError(f"Label {label} is out of range for matrix with {matrix.shape[0]} classes")
    TP = matrix[index, index]
    FN = np.sum(matrix[index, :]) - TP
    denom = TP + FN
    if denom == 0:
        return 0.0
    return TP / denom

In [27]:
def getF1PerClass(matrix, label):
    precision = getPrecisionPerClass(matrix, label)
    recall = getRecallPerClass(matrix, label)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

In [28]:
def getMacroPrecision(matrix):
    num_classes = matrix.shape[0]
    precisions = [getPrecisionPerClass(matrix, label) for label in range(1, num_classes + 1)]
    return np.mean(precisions)

In [29]:
def getMacroRecall(matrix):
    num_classes = matrix.shape[0]
    recalls = [getRecallPerClass(matrix, label) for label in range(1, num_classes + 1)]
    return np.mean(recalls)

In [30]:
def getMacroF1(matrix):
    num_classes = matrix.shape[0]
    f1_scores = [getF1PerClass(matrix, label) for label in range(1, num_classes + 1)]
    return np.mean(f1_scores)

# **compare the two approaches (the implemented one and the scikit-learn one)**

In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Construct the implmented confusion matrix
implenentedClassfierMatrix = confusion_matrix(test_labels, FinalPrediction)
# Construct the scikit-learn confusion matrix
sklearnClassfierMatrix = confusion_matrix(test_labels, predictions)

print("Scikit-learn Confusion Matrix using MultinomialNB:")
print(sklearnClassfierMatrix)

print("Scikit-learn Confusion Matrix using our implementation:")
print(implenentedClassfierMatrix)

# Construct the implmented confusion matrix
implenentedClassfierMatrix = im_confusion_matrix(test_labels, FinalPrediction)
# Construct the scikit-learn confusion matrix
sklearnClassfierMatrix = im_confusion_matrix(test_labels, predictions)

print("Implemented Confusion Matrix using MultinomialNB :")
print(sklearnClassfierMatrix)

print("Implemented Confusion Matrix our implementation:")
print(implenentedClassfierMatrix)


Scikit-learn Confusion Matrix using MultinomialNB:
[[ 23 206   9  40   1]
 [ 21 414  57 132   9]
 [  8 162  32 179   8]
 [  2  91  36 337  44]
 [  1  37  11 262  88]]
Scikit-learn Confusion Matrix using our implementation:
[[ 30 193  11  44   1]
 [ 20 413  58 135   7]
 [  9 162  31 177  10]
 [  2  87  31 350  40]
 [  2  38  14 267  78]]
{0.0, 1.0, 2.0, 3.0, 4.0}
{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0}
Implemented Confusion Matrix using MultinomialNB :
[[414.  57. 132.   9.   0.]
 [162.  32. 179.   8.   0.]
 [ 91.  36. 337.  44.   0.]
 [ 37.  11. 262.  88.   0.]
 [  0.   0.   0.   0.   0.]]
Implemented Confusion Matrix our implementation:
[[413.  58. 135.   7.   0.]
 [162.  31. 177.  10.   0.]
 [ 87.  31. 350.  40.   0.]
 [ 38.  14. 267.  78.   0.]
 [  0.   0.   0.   0.   0.]]


In [37]:
# trying our accuracy_score, precision_score, recall_score, f1_score implmentation  vs scikit-learn
implenentedClassfierMatrix = confusion_matrix(test_labels, FinalPrediction)
# Construct the scikit-learn confusion matrix
sklearnClassfierMatrix = confusion_matrix(test_labels, predictions)

classes = [0,1,2,3,4]
for c in classes:
  print(f"Class {c}:")
  print(f"Implemented Precision: {getPrecisionPerClass(sklearnClassfierMatrix, c)}")
  print(f"Scikit-learn Precision: {precision_score(test_labels, predictions, average=None)[c]}")
  print(f"Implemented Recall: {getRecallPerClass(sklearnClassfierMatrix, c)}")
  print(f"Scikit-learn Recall: {recall_score(test_labels, predictions, average=None)[c]}")
  print(f"Implemented F1-Score: {getF1PerClass(sklearnClassfierMatrix, c)}")
  print(f"Scikit-learn F1-Score: {f1_score(test_labels, predictions, average=None)[c]}")
  print("----------------------------------------------")






Class 0:


ValueError: Label 0 is out of range for matrix with 5 classes